<a href="https://colab.research.google.com/github/jelegend/ANN-MLsummerproject/blob/master/MNIST_Classification_Model_%5BGPU%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn import functional
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import time
from IPython import display

In [0]:
# grayscale and inline plotting
%matplotlib inline
plt.rcParams['image.cmap'] = 'gray'

In [0]:
def plot_image(image):
    nr, nc = image.shape
    extent = [-0.5, nc - 0.5, nr - 0.5, -0.5]
    plt.imshow(image, extent=extent, origin='upper', interpolation='nearest')

def visualize(t, loss, errcl, x5, x4, x3, x0, w1):

    loss_avg = np.divide(
        np.cumsum(loss[: t + 1]),
        range(1, t + 2)
    )

    errcl_avg = np.divide(
        np.cumsum(errcl[: t + 1]),
        range(1, t + 2)
    )

    n_last_batches = np.min([20, t])
    k = np.ones(n_last_batches * 2 + 1) / (n_last_batches + 1)
    k[:n_last_batches] = 0
    
    errcl_sm = np.convolve(np.pad(errcl, mode="edge", pad_width=n_last_batches), k, mode="valid")
    errcl_sm = errcl_sm[: len(errcl_avg)]

    loss_sm = np.convolve(np.pad(loss, mode="edge", pad_width=n_last_batches), k, mode="valid")
    loss_sm = loss_sm[: len(loss)]

    display.clear_output(wait=True)

    plt.subplot(4, 3, 1)
    plt.plot(loss, label="loss")
    plt.plot(loss_sm, label="smothed loss")
    plt.plot(loss_avg, label="avg loss")
    plt.legend()
    plt.ylim(0, np.max(loss)*1.05)
    plt.title("loss: avg - %.4f, smoothed - %.4f, current - %.4f"  % (loss_avg[t], loss_sm[t], loss[t]))

    plt.subplot(4, 3, 2)
    plt.plot(errcl, label="cl err")
    plt.plot(errcl_sm, label="smothed cl err")
    plt.plot(errcl_avg, label="avg cl err")
    plt.legend()
    plt.ylim(0, np.max(errcl)*1.05)
    plt.title("cl error: avg - %.4f, smoothed - %.4f, current - %.4f"  % (errcl_avg[t], errcl_sm[t], errcl[t]))
    
    plt.subplot(4, 3, 3)
    plt.bar(range(len(x5)), x5)
    plt.title("class confidences")

    plt.subplot(4,3,4)
    plt.hist(x4)
    plt.title("F6 activations")

    plt.subplot(4,3,5)
    plt.hist(x3)
    plt.title("C5 activations")

    plt.subplot(4,3,6)
    plot_image(x0)
    plt.title("input image")

    for i in range(w1.shape[0]):
        plt.subplot(4,3,7+i)
        plot_image(w1[i])
        plt.title("C1 kernel channel " + str(i))

    plt.subplots_adjust(wspace=0.5)
    plt.subplots_adjust(hspace=0.5)
    plt.gcf().set_size_inches(18.5, 10.5)
    display.display(plt.gcf())

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])


mnist = torchvision.datasets.MNIST(root='data', train=True, download=True, transform=transform) # train data only
trainimages = mnist.data
trainlabels = mnist.targets



trainloader = torch.utils.data.DataLoader(mnist, batch_size=32, shuffle=True)

# check training data shape
print ("Training Data shape is: ", list(trainimages.size()))
print ("Training Target shape is: ", list(trainlabels.size()))

Training Data shape is:  [60000, 28, 28]
Training Target shape is:  [60000]


In [0]:
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32, kernel_size=(3,3), stride=1)
        # convolutional layer
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=(3,3), stride=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(kernel_size = (2,2),stride=2)
        # dropout layer 1 (p=0.25)
        self.dropout1 = nn.Dropout(0.25)
        # linear layer (9216 -> 128)
        self.fc1 = nn.Linear(9216, 128)
        # dropout layer 2 (p=0.5)
        self.dropout2 = nn.Dropout(0.5)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(128, 10)

        
    def forward(self, x):
       # add sequence of convolutional and max pooling layers
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        #Flattening the output for the Linear layers
        x = x.view(x.size(0),-1) 
        #Dropout Layer
        x = self.dropout1(x)
        #Linear fully-connected layer and ReLU
        x = torch.relu(self.fc1(x))
        #Dropout Layer
        x = self.dropout2(x)
        #Linear fully-connected layer and Softmax
        x = torch.softmax(self.fc2(x), dim=1)
        
        return x

net = Network()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [0]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')